In [1]:
import pandas as pd
import os
import gc
import numpy as np

from GLC.data_loading.common import load_patch
from GLC.metrics import top_30_error_rate

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, subset='train'):
        data = pd.read_csv("../data/observations/observations_fr_train.csv", sep=";", index_col="observation_id")
        ind = data.index[data["subset"] == subset]
        data = data.loc[ind]
        self.observation_ids = data.index
        self.labels = self.img_labels = data.species_id.values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        to_tensor = []
        to_tensor.append(load_patch(self.observation_ids[index], "../data", data="near_ir")[0])
        to_tensor.append(load_patch(self.observation_ids[index], "../data", data="rgb")[0][:,:,2])
        to_tensor.append(load_patch(self.observation_ids[index], "../data", data="rgb")[0][:,:,0])
        to_tensor = np.array(to_tensor)
        return torch.from_numpy(to_tensor).float(), self.labels[index]

In [3]:
train_dataset = CustomImageDataset('train')
test_dataset = CustomImageDataset('val')

In [4]:
train_loader = DataLoader(train_dataset, batch_size=32,num_workers=0,shuffle = True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0, shuffle = False, drop_last=True)

In [5]:
n_classes = 17036

model = models.resnet50(pretrained=True)
model.fc = nn.Sequential(
               nn.Linear(2048, 4096),
               nn.ReLU(inplace=True),
              nn.Linear(4096, n_classes))

model = model.cuda()

if os.path.isfile('model/GeoLife_2.pt'):
    print("Load model...")
    model.load_state_dict(torch.load('model/GeoLife_2.pt'))

In [6]:
def train(epoch, model, train_loader, optimizer, loss_fn):
    model.train()
    for j in range(0, epoch):
        for i, data in enumerate(train_loader):
            inputs, targets = data
            
            inputs = inputs.float().cuda()
            targets = targets.cuda()
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
        
            loss = loss_fn(outputs, targets)
            loss.backward()
        
            optimizer.step()
    model.eval()
    return model

In [7]:
gc.collect()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
loss_fn = nn.CrossEntropyLoss()
train(1, model, train_loader, optimizer, loss_fn)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
torch.save(model.state_dict(), "model/GeoLife_3.pt")

In [10]:
model.eval()
result = []

for i, data in enumerate(test_loader):
        inputs, labels = data
        labels = labels.cuda()
        inputs = inputs.float()
        inputs = inputs.cuda()
        outputs = model(inputs)
        
        result.append(top_30_error_rate(labels.cpu(), outputs.cpu().detach()))
        
print("Fehlerrate: ", np.mean(result))

Fehlerrate:  0.7592147435897436
